# Data Splitting

In this notebook I'm splitting and saving the data for sentiment analysis

## Imports

In [1]:
import gc
import os

In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split

In [3]:
SEED = 42

## Paths

In [4]:
relative_path = os.path.join("../", "data")

## Data

### Loading data

In [8]:
reviews_df = pd.read_parquet(os.path.join(relative_path, "reviews_short.parquet"))
reviews_df["review"] = reviews_df["review"].str.replace("<p>", " ")

### Dataset overview

In [10]:
reviews_df.sample(n=10, random_state=SEED)

,show_id,sentiment,review
31211,579,positive,"Киновселенная DC, которая довольно серьезный к..."
72046,98,positive,"Не всякая карта расскажет тебе, чем богато зве..."
12824,217,positive,Этот фильм долго пылился на жестком диске и вч...
203242,742,positive,Что могу сказать... Никогда не любила сериал п...
199423,643,positive,О сериале 'Реальные пацаны' узнал еще летом. К...
125237,844,positive,"Нюквисту – «Оскар»! (Тому самому, из шведской ..."
165963,495,positive,Странные я испытал эмоции при просмотре этой л...
61509,263,positive,В сравнение с другими гангстерскими лентами Ск...
132535,63,positive,"А знаете, этот фильм стоит смотреть одному. Чт..."
60474,706,positive,"Итак, для начала статистика. Данная картина бы..."


### Looking at reviews

In [11]:
for review in reviews_df.sample(n=10, random_state=SEED)["review"].values[:3]:
    print(review.replace("<p>", "\n"))
    print("\n")

Киновселенная DC, которая довольно серьезный конкурент Marvel выпустила очередной фильм. Фильм очень динамичный и насыщен экшеном. Есть моменты, которые не особо понятны, например откуда взялась царица 'Атланты', почему ее выкинуло на берег, откуда взялись пираты на подводной лодке. Непонятно, почему многие пишут, что сценарий слабый, все очень логично и понятно - Царь полукровка 'Аквамен', который должен объединить море и сушу, на это он и полукровка, что среднее между двумя мирами! Также замечу, что и реально отличный юмор присутствует, а юмор отлично разбавляет нудное общение, как и бывает во многих фильмах. Спецэффекты очень качественные, но и присутствует природная красота городов на берегу океана, что сразу хочется оказаться там. Конечно хочется, чтобы не все в фильмах было так прекрасно как здесь, но и какой-нибудь негатив оставался! В целом фильм на 9, не без недостатков, но опустим их и округлим число до 10.


Не всякая карта расскажет тебе, чем богато звездное небо, оно может

In the previous step I've removed the scores from the reviews, so it is now safe to continue with baseline model creation.

### Selecting needed columns

In [12]:
class_col = "label"
text_col = "text"

For baseline model we're interested only in `sentiment` and `review_body` columns

In [13]:
df = reviews_df[["sentiment", "review"]]
df = df.rename(columns={"sentiment":class_col, "review":text_col})

In [14]:
del reviews_df
gc.collect()

308

### Splitting the data

In [15]:
train_df, test_dev_df = train_test_split(
    df, test_size=0.15, random_state=SEED, stratify=df[class_col]
)

test_df, dev_df = train_test_split(
    test_dev_df, test_size=0.5, random_state=SEED, stratify=test_dev_df[class_col]
)

train_df.shape, test_df.shape, dev_df.shape

((175556, 2), (15490, 2), (15491, 2))

In [16]:
train_df[class_col].value_counts(normalize=True)

label
positive    0.720272
neutral     0.150026
negative    0.129702
Name: proportion, dtype: float64

In [17]:
test_df[class_col].value_counts(normalize=True)

label
positive    0.720271
neutral     0.150032
negative    0.129697
Name: proportion, dtype: float64

In [18]:
dev_df[class_col].value_counts(normalize=True)

label
positive    0.720289
neutral     0.150023
negative    0.129688
Name: proportion, dtype: float64

## Saving data

In [19]:
LABEL_MAP = {"negative": 0, "neutral": 1, "positive": 2}

## Whole Dataset

In [20]:
train_df["fold"] = "train"
test_df["fold"] = "test"
dev_df["fold"] = "dev"

In [21]:
data = pd.concat([train_df, test_df, dev_df]).reset_index(drop=True)
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 206537 entries, 0 to 206536
Data columns (total 3 columns):
 #   Column  Non-Null Count   Dtype   
---  ------  --------------   -----   
 0   label   206537 non-null  category
 1   text    206537 non-null  object  
 2   fold    206537 non-null  object  
dtypes: category(1), object(2)
memory usage: 3.3+ MB


In [23]:
data.to_parquet(os.path.join(relative_path, "split_reviews.parquet"))